# 说明
在第2章中，我们仍然解释可以研究什么，而在第3章中，我们展示如何去做。尽管如此，对于某些图表，我使用了代码来收集数据。这些代码可以在这里找到。如果您正在寻找代码并只想开始，最好从第3章的代码开始。

**此版本使用 Tushare 处理中国股票，而不是 yfinance**
**改进版本：解决了 current_ratio 和 quick_ratio 数据不可用的问题**

## 2.2 - 如何获取行业分类

In [ ]:
import tushare as ts
import pandas as pd
import os

# 配置matplotlib中文字体
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings('ignore')

# 检查并设置中文字体
def setup_chinese_font():
    """设置matplotlib支持中文显示"""
    try:
        # 尝试设置中文字体
        # 方法1: 尝试常见的中文字体
        plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'Microsoft YaHei', 'PingFang SC']
        plt.rcParams['axes.unicode_minus'] = False
        
        # 测试字体是否可用
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, '测试中文显示', fontsize=14)
        plt.close(fig)
        print("中文字体配置成功")
        return True
    except Exception as e:
        print(f"中文字体配置警告: {e}")
        # 如果字体设置失败，使用英文
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False
        return False

# 设置字体
font_setup = setup_chinese_font()

# 设置tushare token
ts.set_token(os.getenv('TUSHARE_TOKEN'))
pro = ts.pro_api()

### 2.2.2 行业与经济周期
这是图 2.6 的代码

### 收集比率方法 - 改进版本
解决 current_ratio 和 quick_ratio 数据不可用问题

In [ ]:
def get_balance_sheet_data(code, period='20231231'):
    """
    获取资产负债表数据
    
    Parameters:
        code (str): 股票代码
        period (str): 报告期，格式为YYYYMMDD
    
    Returns:
        DataFrame: 资产负债表数据
    """
    try:
        # 获取资产负债表数据
        balance_sheet = pro.balancesheet(ts_code=code, period=period, fields='ts_code,end_date,total_assets,total_liab,total_cur_assets,total_cur_liab,cash_and_equivalents_at_end,notes_receivable,inventory')
        return balance_sheet
    except Exception as e:
        print(f"获取 {code} 资产负债表数据失败: {e}")
        return pd.DataFrame()

def calculate_liquidity_ratios(code, period='20231231'):
    """
    计算流动比率和速动比率
    
    Parameters:
        code (str): 股票代码
        period (str): 报告期
    
    Returns:
        dict: 包含流动比率和速动比率的字典
    """
    try:
        # 获取资产负债表数据
        balance_data = get_balance_sheet_data(code, period)
        
        if balance_data.empty:
            return {'current_ratio': None, 'quick_ratio': None}
        
        latest_data = balance_data.iloc[0]
        
        # 获取关键数据
        total_cur_assets = latest_data.get('total_cur_assets')  # 流动资产合计
        total_cur_liab = latest_data.get('total_cur_liab')      # 流动负债合计
        cash_and_equivalents = latest_data.get('cash_and_equivalents_at_end')  # 货币资金
        notes_receivable = latest_data.get('notes_receivable')  # 应收账款
        inventory = latest_data.get('inventory')               # 存货
        
        # 计算流动比率 = 流动资产 / 流动负债
        current_ratio = None
        if total_cur_assets and total_cur_liab and total_cur_liab != 0:
            current_ratio = round(total_cur_assets / total_cur_liab, 4)
        
        # 计算速动比率 = (流动资产 - 存货) / 流动负债
        # 或者更精确的：(货币资金 + 应收账款) / 流动负债
        quick_ratio = None
        if total_cur_assets and total_cur_liab and total_cur_liab != 0:
            # 方法1：使用流动资产 - 存货
            if inventory:
                quick_assets = total_cur_assets - inventory
                quick_ratio = round(quick_assets / total_cur_liab, 4)
            # 方法2：如果有更详细的数据，使用货币资金 + 应收账款
            elif cash_and_equivalents and notes_receivable:
                quick_assets = cash_and_equivalents + notes_receivable
                quick_ratio = round(quick_assets / total_cur_liab, 4)
        
        return {
            'current_ratio': current_ratio,
            'quick_ratio': quick_ratio,
            'total_cur_assets': total_cur_assets,
            'total_cur_liab': total_cur_liab,
            'inventory': inventory
        }
        
    except Exception as e:
        print(f"计算 {code} 流动性比率失败: {e}")
        return {'current_ratio': None, 'quick_ratio': None}

def get_improved_stock_financial_data(codes):
    """
    获取改进的股票财务数据（包含手动计算的流动性比率）
    
    Parameters:
        codes (list): 股票代码列表
    
    Returns:
        DataFrame: 包含财务数据的DataFrame
    """
    data_list = []
    
    for code in codes:
        try:
            # 获取基本信息
            basic_info = pro.stock_basic(ts_code=code, fields='ts_code,symbol,name,area,industry,list_date')
            
            # 获取财务指标数据
            try:
                finance_data = pro.fina_indicator(ts_code=code, period='20231231')
                
                # 计算流动性比率
                liquidity_data = calculate_liquidity_ratios(code)
                
                if not finance_data.empty:
                    latest_data = finance_data.iloc[0]
                    
                    # 合并数据
                    row = {
                        'code': code,
                        'name': basic_info.iloc[0]['name'],
                        'industry': basic_info.iloc[0]['industry'],
                        'area': basic_info.iloc[0]['area'],
                        # 流动性比率（手动计算）
                        'current_ratio': liquidity_data['current_ratio'],
                        'quick_ratio': liquidity_data['quick_ratio'],
                        # 传统财务指标
                        'debt_to_assets': latest_data.get('debt_to_assets'),  # 资产负债率
                        'roe': latest_data.get('roe'),  # 净资产收益率
                        'net_profit_margin': latest_data.get('net_profit_margin'),  # 净利润率
                    }
                    data_list.append(row)
                    
            except Exception as e:
                print(f"无法获取 {code} 的财务数据: {e}")
                
        except Exception as e:
            print(f"无法获取 {code} 的基本信息: {e}")
    
    return pd.DataFrame(data_list)

## 流动性比率 - 改进版本
表 2.4 和表 2.5

In [ ]:
# 示例：获取股票财务数据
stock_codes = ['000002.SZ', '600048.SH', '002415.SZ']  # 万科A, 保利发展, 海康威视
financial_data = get_improved_stock_financial_data(stock_codes)
print(financial_data)

## 银行业特殊说明

In [ ]:
def explain_bank_liquidity_ratios():
    """
    解释银行股的流动性比率特殊性
    """
    print("=== 银行股流动性比率的特殊性 ===\n")
    
    print("1. 银行业的特殊性质:")
    print("   - 银行的主要业务是吸收存款和发放贷款")
    print("   - 存款是银行的负债，贷款是银行的主要资产")
    print("   - 传统的流动性比率对银行业不太适用\n")
    
    print("2. 银行流动比率的特点:")
    print("   - 通常 < 1.0，因为贷款远大于现金")
    print("   - 低流动比率不一定意味着风险高")
    print("   - 反映银行的资金运用效率\n")
    
    print("3. 银行业更重要的指标:")
    print("   - 资本充足率 (Capital Adequacy Ratio)")
    print("   - 存贷比 (Loan-to-Deposit Ratio)")
    print("   - 不良贷款率 (NPL Ratio)")
    print("   - 拨备覆盖率 (Provision Coverage Ratio)\n")
    
    print("4. 如何正确解读银行股的财务数据:")
    print("   - 不要用传统行业的标准判断银行业")
    print("   - 关注监管指标和风险控制能力")
    print("   - 考虑宏观经济环境和政策影响")

# 执行说明
explain_bank_liquidity_ratios()

### 盈利

In [ ]:
# 计算市盈率 (PE)
def get_pe_ratio(code):
    """
    获取并计算市盈率
    """
    try:
        # 获取股票基本信息
        basic_info = pro.stock_basic(ts_code=code, fields='ts_code,trade_status')
        
        # 获取当前股价
        daily_data = pro.daily(ts_code=code, start_date='20231201', end_date='20231231')
        current_price = daily_data.iloc[0]['close'] if not daily_data.empty else None
        
        # 获取每股收益 (EPS)
        finance_data = pro.fina_indicator(ts_code=code, period='20231231')
        eps = finance_data.iloc[0]['eps'] if not finance_data.empty else None
        
        if current_price and eps and eps != 0:
            pe = round(current_price / eps, 2)
            return pe
        else:
            return None
            
    except Exception as e:
        print(f"获取 {code} PE比率失败: {e}")
        return None

# 估值

In [ ]:
def get_valuation_data(codes):
    """
    获取估值指标数据
    """
    data_list = []
    
    for code in codes:
        try:
            # 获取基本信息
            basic_info = pro.stock_basic(ts_code=code, fields='ts_code,symbol,name,industry')
            
            # 获取财务指标
            finance_data = pro.fina_indicator(ts_code=code, period='20231231')
            
            # 获取股价数据
            daily_data = pro.daily(ts_code=code, start_date='20231201', end_date='20231231')
            current_price = daily_data.iloc[0]['close'] if not daily_data.empty else None
            
            if not finance_data.empty:
                latest = finance_data.iloc[0]
                
                # 计算各种估值指标
                pe = None
                if latest.get('eps') and latest.get('eps') != 0 and current_price:
                    pe = round(current_price / latest.get('eps'), 2)
                
                pb = latest.get('pb')  # 市净率
                ps = latest.get('ps')  # 市销率
                
                row = {
                    'code': code,
                    'name': basic_info.iloc[0]['name'],
                    'industry': basic_info.iloc[0]['industry'],
                    'PE': pe,
                    'PB': pb,
                    'PS': ps
                }
                data_list.append(row)
        except Exception as e:
            print(f"获取 {code} 数据失败: {e}")
    
    return pd.DataFrame(data_list)

In [ ]:
# 示例：科技股估值对比
tech_codes = ['000858.SZ', '002415.SZ', '000002.SZ']  # 五粮液, 海康威视, 万科A
valuation_data = get_valuation_data(tech_codes)
print(valuation_data)

## 股息
表 2.10

In [ ]:
def get_dividend_data(codes):
    """
    获取股息数据
    """
    data_list = []
    
    for code in codes:
        try:
            # 获取基本信息
            basic_info = pro.stock_basic(ts_code=code, fields='ts_code,symbol,name,industry')
            
            # 获取分红数据
            dividend_data = pro.dividend(ts_code=code, fields='ts_code,end_date,divi,bonus_share')
            
            # 计算股息率
            if not dividend_data.empty:
                # 获取最新股价
                daily_data = pro.daily(ts_code=code, start_date='20231201', end_date='20231231')
                current_price = daily_data.iloc[0]['close'] if not daily_data.empty else None
                
                # 获取最新分红
                latest_dividend = dividend_data.iloc[0]
                divi = latest_dividend.get('divi', 0)
                
                if divi and current_price and current_price != 0:
                    dividend_yield = (divi / current_price) * 100  # 股息率 (%)
                else:
                    dividend_yield = None
                
                row = {
                    'code': code,
                    'name': basic_info.iloc[0]['name'],
                    'industry': basic_info.iloc[0]['industry'],
                    'dividend_per_share': divi,
                    'dividend_yield': dividend_yield
                }
                data_list.append(row)
        except Exception as e:
            print(f"获取 {code} 股息数据失败: {e}")
    
    return pd.DataFrame(data_list)

# 股权

In [ ]:
def get_market_cap(code):
    """
    获取市值和流通股本
    """
    try:
        # 获取市值数据
        daily_data = pro.daily(ts_code=code, start_date='20231201', end_date='20231231')
        current_price = daily_data.iloc[0]['close'] if not daily_data.empty else None
        
        # 获取股本数据
        finance_data = pro.fina_indicator(ts_code=code, period='20231231')
        shares_outstanding = finance_data.iloc[0]['shares_outstanding'] if not finance_data.empty else None
        
        if current_price and shares_outstanding:
            market_cap = current_price * shares_outstanding * 100000000  # 转换为元
            print(f"{code} 市值: {market_cap/100000000:.2f} 亿元")
            return market_cap
        else:
            print(f"{code}: 数据不可用")
            return None
    except Exception as e:
        print(f"获取 {code} 市值失败: {e}")
        return None

# 桑基图 - 公司收入和支出流向

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.sankey import Sankey

# 确保中文字体设置
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'Microsoft YaHei', 'PingFang SC']
plt.rcParams['axes.unicode_minus'] = False

# 创建一个新的图形和坐标轴
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[],
                     title="公司收入和支出流向图")

# 创建桑基图
sankey = Sankey(ax=ax, scale=0.01, offset=0.2, head_angle=180,
                format='%.0f', unit='%')

# 添加流向
sankey.add(flows=[30, 20, 50, -40, -20, -40],
           labels=['主营业务收入', '其他收入', '投资收益', '生产成本', '研发支出', '运营费用'],
           orientations=[0, 1, -1, 1, 1, -1],
           pathlengths=[0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
           patchlabel="收入",
           facecolor='#377eb8')

# 完成图形
diagrams = sankey.finish()

# 设置透明度
for diagram in diagrams:
    diagram.patch.set_alpha(0.5)

# 保存图片
plt.savefig("sankey_diagram_chinese.png", dpi=300, bbox_inches='tight')
plt.show()

## 总结

本文件展示了如何使用 Tushare 获取中国股票的基本财务数据，包括：

1. **流动性比率** - 通过资产负债表手动计算
2. **盈利能力** - ROE、ROA等指标
3. **估值指标** - PE、PB、PS等
4. **股息数据** - 分红和股息率
5. **股权信息** - 市值和股本

主要改进：解决了 `current_ratio` 和 `quick_ratio` 数据不可用的问题，通过手动计算流动性比率来获取完整的财务分析数据。